In [1]:
import pyspark

import findspark
findspark.init()

from pyspark.sql import *

from pyspark import SparkContext
sc = SparkContext.getOrCreate();              # It creates a new SparkContext to run our applications in it.
sc

from pyspark.sql import SparkSession

# Creating a SparkSession
spark = SparkSession.builder.appName("MyApppp").getOrCreate()

In [2]:
rdd = sc.textFile('sharemarket.csv')
schema = ["MARKET","SERIES","SYMBOL","SECURITY","PREV_CL_PR","OPEN_PRICE","HIGH_PRICE", \
          "LOW_PRICE","CLOSE_PRICE","NET_TRDVAL","NET_TRDQTY","CORP_IND","TRADES","HI_52_WK","LO_52_WK"]

rdd = rdd.map(lambda x: x.split(","))
rdd.collect()



df = spark.createDataFrame(rdd,schema =schema)
df.show(2)


+------+------+------+--------------------+----------+----------+----------+---------+-----------+----------+----------+--------+------+--------+--------+
|MARKET|SERIES|SYMBOL|            SECURITY|PREV_CL_PR|OPEN_PRICE|HIGH_PRICE|LOW_PRICE|CLOSE_PRICE|NET_TRDVAL|NET_TRDQTY|CORP_IND|TRADES|HI_52_WK|LO_52_WK|
+------+------+------+--------------------+----------+----------+----------+---------+-----------+----------+----------+--------+------+--------+--------+
|     N|    N1|  IRFC|BOND 8.00% PA TAX...|      1086|      1085|      1085|  1084.54|       1085|3371959.44|      3108|        |     8|    1194|    1050|
|     N|    N1|  JNPT|BOND 6.82% PA TAX...|      1001|    1007.1|      1015|   1007.1|       1015|     51370|        51|        |     2| 1529.99|    1000|
+------+------+------+--------------------+----------+----------+----------+---------+-----------+----------+----------+--------+------+--------+--------+
only showing top 2 rows



# 1.Query to display the number of series present in the data.

In [3]:
share = df.registerTempTable("Sales")
spark.sql("select count(distinct series) from sales").show()

c:\python38\lib\site-packages\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+----------------------+
|count(DISTINCT series)|
+----------------------+
|                    53|
+----------------------+



# 2.Display the series present in the data.

In [4]:
spark.sql("select distinct series from sales").show()

+------+
|series|
+------+
|    NA|
|    N3|
|    NC|
|    N7|
|    N8|
|    N2|
|    N5|
|    N4|
|    N6|
|    N1|
|    N9|
|    NB|
|    YH|
|    NS|
|    NL|
|    NK|
|    Z4|
|    NJ|
|    NX|
|    YS|
+------+
only showing top 20 rows



# 3.Find the sumpof all the prices in the each series.

In [5]:
spark.sql("select series, sum(open_price + high_price + low_price + close_price) Total_Price from Sales group by series").show()

+------+------------------+
|series|       Total_Price|
+------+------------------+
|    NA|          79628.91|
|    N3| 8348.119999999999|
|    NC|          57412.31|
|    N7|13045.900000000001|
|    N8|32583.230000000003|
|    N2|          42048.85|
|    N5|          35092.07|
|    N4|47750.659999999996|
|    N6|         105463.81|
|    N1|          21630.72|
|    N9|           32401.0|
|    NB|          28642.17|
|    YH|           4098.02|
|    NS|            4129.0|
|    NL|13611.080000000002|
|    NK|7731.4800000000005|
|    Z4|            4124.2|
|    NJ|          17155.18|
|    NX|           1275.88|
|    YS|            4530.0|
+------+------------------+
only showing top 20 rows



# 4.Display security,series with highest net trade value

In [6]:
# Maximum net_trdval in each Series.

# spark.sql("select security, series, max(net_trdval) from sales group by security,series order by 1").show()

# For maximum net_trd value.

spark.sql("select security, series, net_trdval from sales where net_trdval = (select max(net_trdval) from sales)").show()

+--------------------+------+----------+
|            security|series|net_trdval|
+--------------------+------+----------+
|SEC RED NCD 9.75 ...|    N4|  972888.7|
+--------------------+------+----------+



# 5.Display the series whose sum of all prices greater than the net trade value.

In [7]:
spark.sql("select series,(open_price + high_price + low_price + close_price) TotalPrice, net_trdval from sales \
           where (open_price + high_price + low_price + close_price)>net_trdval").show()


+------+----------+----------+
|series|TotalPrice|net_trdval|
+------+----------+----------+
|    N2|    4888.0|      3666|
|    N4|   4088.04|   2044.02|
|    N5|    3824.8|    1912.4|
|    N6|    5364.0|      1341|
|    NB|    3686.0|      1843|
+------+----------+----------+



# 6.Display the series with highest net trade quantity.

In [8]:
spark.sql("select series, net_trdqty from sales where net_trdqty = (select max(float(net_trdqty)) from sales)").show()

+------+----------+
|series|net_trdqty|
+------+----------+
|    N3|    143810|
+------+----------+



# 7. Display the highest and lowest open price.

In [11]:
spark.sql("select max(float(open_price)) Highest ,min(float(open_price)) Lowest from sales").show()

+-------+------+
|Highest|Lowest|
+-------+------+
|11450.0| 13.75|
+-------+------+



# 8.Query to display the series which have trades more than 80.

In [13]:
spark.sql("select series from sales where float(trades)>80 ").show()

+------+
|series|
+------+
|    N2|
|    N3|
|    N7|
+------+



# 9.Display the difference between the net trade value net trade quantity for each series.

In [15]:
spark.sql("select series, sum(net_trdval - net_trdqty) from sales group by series").show()

+------+------------------------------+
|series|sum((net_trdval - net_trdqty))|
+------+------------------------------+
|    NA|                    3331953.97|
|    N3|                    5894945.91|
|    NC|                    2505038.79|
|    N7|                    2218463.04|
|    N8|                    3982020.85|
|    N2|                    3674571.25|
|    N5|                    5581855.67|
|    N4|            3474029.2199999997|
|    N6|                 1.678578209E7|
|    N1|                    7499601.93|
|    N9|                      455370.9|
|    NB|                    2430891.59|
|    YH|                       61270.1|
|    NS|                      354013.9|
|    NL|                    1581525.31|
|    NK|                    1010121.69|
|    Z4|                       36956.6|
|    NJ|                    7421503.24|
|    NX|                      12728.72|
|    YS|                       79055.0|
+------+------------------------------+
only showing top 20 rows

